In [178]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import numpy as np
from sklearn import preprocessing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
from os.path import exists as DoesPathExist
train_logs_path = "/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv"
train_scores_path = "/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv"
test_logs_path = "/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv"
sample_submission_path = "/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv"

if DoesPathExist(train_logs_path):
    train_logs = pd.read_csv(train_logs_path)
else:
    train_logs = pd.read_csv("./data/train_logs.csv")

if DoesPathExist(train_scores_path):
    train_scores = pd.read_csv(train_scores_path)
else:
    train_scores = pd.read_csv("./data/train_scores.csv")

if DoesPathExist(test_logs_path):
    test_logs = pd.read_csv(test_logs_path)
else:
    test_logs = pd.read_csv("./data/test_logs.csv")
    
if DoesPathExist(sample_submission_path):
    sample_submission = pd.read_csv(sample_submission_path)
else:
    sample_submission = pd.read_csv("./data/sample_submission.csv")
    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('expand_frame_repr', False)

In [135]:
train_logs.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [136]:
train_scores.head()

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0


In [137]:
train_logs.shape

cols_to_drop = ["down_event", "up_event"]
for col in cols_to_drop:
    train_logs = train_logs.drop(col, axis=1) 
train_logs.shape

(8405898, 9)

In [138]:
def FeatureEngineering(input: pd.DataFrame) -> pd.DataFrame:
    """Does all of the feature engineering desired.

    Args:
        input (pd.DataFrame): Input dataframe to extract features from.

    Returns:
        pd.DataFrame: Dataframe with all of the information
    """
    ids = input["id"].unique()
    all_essays = []

    for id in ids:
        essay = []
        essay.append(id)
        raw_essay = input.loc[input["id"] == id]
        num_inputs = raw_essay.loc[raw_essay["activity"] == "Input"].shape[0]
        num_removals = raw_essay.loc[raw_essay["activity"] == "Remove/Cut"].shape[0]
        word_count = raw_essay.max()["word_count"]
        avg_action_time = raw_essay[["action_time"]].mean()
        all_essays.append((id, num_inputs, num_removals, word_count, avg_action_time))
    
    return pd.DataFrame(all_essays, columns=["id", "num_inputs", "num_removals", "word_count", "avg_action_time"])

train_data = FeatureEngineering(train_logs)

In [139]:
def NormalizeTrainData(input: pd.DataFrame) -> pd.DataFrame:
    min_max_scaler = preprocessing.MinMaxScaler()
    input["num_inputs"] = min_max_scaler.fit_transform(input[["num_inputs"]])
    input["num_removals"] = min_max_scaler.fit_transform(input[["num_removals"]])
    input["word_count"] = min_max_scaler.fit_transform(input[["word_count"]])
    input["avg_action_time"] = min_max_scaler.fit_transform(input[["avg_action_time"]])
    return input

train_data = NormalizeTrainData(train_data)

/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/_array_api.py:380: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  array = numpy.asarray(array, order=order, dtype=dtype)
/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/_array_api.py:380: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  array = numpy.asarray(array, order=order, dtype=dtype)


In [140]:
def OneHotEncoding(n: float):
    result = [0]*12
    result[int(n / 0.5) - 1] = 1
    return result

ids = train_data["id"].unique()
x_train = train_data.sort_values("id").drop("id", axis=1)
raw_scores = train_scores[train_scores["id"].isin(ids)].sort_values("id").drop("id", axis=1)
y_train = np.array([OneHotEncoding(float(x)) for x in raw_scores['score']])

In [141]:
num_train = len(y_train)
x_valid = x_train[:int(num_train*0.2)]
y_valid = y_train[:int(num_train*0.2)]
x_train = x_train[int(num_train*0.2):]
y_train = y_train[int(num_train*0.2):]

In [142]:
model = Sequential([
    Dense(units=512, activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=12, activation="sigmoid")
])

In [147]:
predictions = model(np.array(x_train)).numpy()
predictions
tf.nn.softmax(predictions).numpy()
loss_fn = tf.keras.losses.CategoricalCrossentropy()
loss_fn(y_train, predictions).numpy()
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, verbose=2)

Epoch 1/100
62/62 - 0s - loss: 1.7618 - accuracy: 0.3101 - 177ms/epoch - 3ms/step
Epoch 2/100
62/62 - 0s - loss: 1.7555 - accuracy: 0.3075 - 45ms/epoch - 733us/step
Epoch 3/100
62/62 - 0s - loss: 1.7447 - accuracy: 0.3192 - 50ms/epoch - 805us/step
Epoch 4/100
62/62 - 0s - loss: 1.7444 - accuracy: 0.3131 - 48ms/epoch - 769us/step
Epoch 5/100
62/62 - 0s - loss: 1.7412 - accuracy: 0.3136 - 48ms/epoch - 772us/step
Epoch 6/100
62/62 - 0s - loss: 1.7443 - accuracy: 0.3060 - 52ms/epoch - 837us/step
Epoch 7/100
62/62 - 0s - loss: 1.7467 - accuracy: 0.3212 - 58ms/epoch - 930us/step
Epoch 8/100
62/62 - 0s - loss: 1.7424 - accuracy: 0.3252 - 53ms/epoch - 849us/step
Epoch 9/100
62/62 - 0s - loss: 1.7421 - accuracy: 0.3151 - 52ms/epoch - 838us/step
Epoch 10/100
62/62 - 0s - loss: 1.7430 - accuracy: 0.3096 - 56ms/epoch - 901us/step
Epoch 11/100
62/62 - 0s - loss: 1.7403 - accuracy: 0.3161 - 56ms/epoch - 904us/step
Epoch 12/100
62/62 - 0s - loss: 1.7382 - accuracy: 0.3156 - 53ms/epoch - 852us/step
Ep

In [150]:
results = model.evaluate(x_valid, y_valid)
for a in range(len(results)):
    print(f"{model.metrics_names[a]}: {results[a]:.3f}")

predictions: np.array = model.predict(x_valid)
scaled_Pred = []
for p in predictions:
    max = p.max()
    scaled_Pred.append(np.array([1 if x == max else 0 for x in p]))


['loss', 'accuracy']
16/16 [==============================] - 0s 535us/step - loss: 1.7785 - accuracy: 0.3178
loss: 1.779
accuracy: 0.318
16/16 [==============================] - 0s 829us/step
[array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]), array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]), array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]), array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0

In [180]:
# Generate the submission
test_data = FeatureEngineering(test_logs)
test_data = NormalizeTrainData(test_data)
predictions = []
for index, row in test_data.iterrows():
    x_data = [x for x in row[1:]]
    pred = model.predict([x_data])
    max = pred[0].max()
    pred = [1 if x == max else 0 for x in pred[0]]
    result = 0.5
    for index, p in enumerate(pred):
        if p == 1:
            result = 0.5 * index + 0.5
    
    predictions.append((pred[0], result)) # (Id, result)
print(predictions)

1/1 [==============================] - 0s 23ms/step
[(0, 1.0), (0, 3.5), (0, 3.5)]


/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/_array_api.py:380: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  array = numpy.asarray(array, order=order, dtype=dtype)
/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/_array_api.py:380: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  array = numpy.asarray(array, order=order, dtype=dtype)


In [183]:
ids = [p[0] for p in predictions]
scores = [p[1] for p in predictions]
sample_submission["id"] = ids
sample_submission["score"] = scores
sample_submission.to_csv("submission.csv")

         id  score
0  0000aaaa    1.0
1  2222bbbb    2.0
2  4444cccc    3.0
Index(['id', 'score'], dtype='object')


ValueError: Length of values (3) does not match length of index (2)